<a href="https://colab.research.google.com/github/kweyuchesa/MORINGA_WEEK_4_IP/blob/master/Moringa_Data_Science_Prep_W4_Independent_Project_2019_07_Kevin_Chesa_Python_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#libraries for our analysis 

In [0]:
## Import the packages and libraries
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import plotly as py
import seaborn as sns 
import os


Importing Our **Autolib_dataset.xlsx** Dataset for analysis and Exploration 

In [106]:
# Let's read the data from the CSV file and create the dataframe to be used
url = 'http://bit.ly/Autolibdataset'
url

'http://bit.ly/Autolibdataset'

#Creating an SQL Environment for Loading Huge Dataset

In [107]:
import os
import csv
import sqlite3
from sqlalchemy import create_engine

csv_database = create_engine('sqlite:////csv_database.db')
chunk_size = 1000000
i = 0
j = 0
for df in pd.read_csv(url, chunksize = chunk_size, iterator = True):
    df = df.rename(columns = {c: c.replace(' ','') for c in df.columns})
    df.index += j

df.to_sql('auto', csv_database, if_exists = 'append' )
j = df.index[-1]+1

print(' | index: {}'.format(j))

 | index: 14421829


In [0]:
#Finding the size of the dataset 
auto.tail()

#Retreaving csv file into our sql lite environment 

DESCRIBING THE DATASET

In [0]:
#retreaving csv file into our sql lite environment 
auto = pd.read_sql_query('SELECT * from auto WHERE city == "Paris" ', csv_database, )

In [0]:
auto.to_csv('auto_clean.csv')

In [0]:
auto = pd.read_csv('auto_clean.csv', dtype = {'year':str,'month':str,'day':str,'hour':str,'minute':str })

In [0]:
auto['date'] = auto.year.str.cat([auto.month,auto.day], sep = '/')

In [0]:
auto['time'] = auto.hour.str.cat([auto.minute], sep = ':')

In [0]:
auto['date_time'] = auto.date.str.cat([auto.time], sep = ' ')

In [0]:
auto.drop(['year','month', 'day', 'minute',  'date', 'time'],axis = 1, inplace = True)

In [0]:
auto.to_csv('auto_clean.csv')

In [0]:
auto.head()

Renaming Our dataset

Preview our dataset

In [92]:
auto.tail()

,Unnamed: 0,Bluecarcounter,Utilibcounter,Utilib1.4counter,City,Postalcode,Publicname,Stationtype,Status,Subscriptionstatus,hour,date_time
690793,690793,3,0,0,Paris,75013,Paris/Auguste Blanqui/50,station,ok,nonexistent,23,2018/4/9 23:59
690794,690794,0,0,0,Paris,75009,Paris/Turgot/4,station,closed,nonexistent,23,2018/4/9 23:59
690795,690795,0,0,0,Paris,75011,Paris/Parmentier/125,station,ok,nonexistent,23,2018/4/9 23:59
690796,690796,0,0,0,Paris,75006,Paris/Danton/8,station,ok,nonexistent,23,2018/4/9 23:59
690797,690797,5,0,0,Paris,75015,Paris/Olivier de Serres/3,station,ok,nonexistent,23,2018/4/9 23:59


#Validity ~ Dropping Unnecesary columns/Variables 

In [0]:
auto.drop(['Unnamed: 0', 'Unnamed:0', 'index',  'Displayedcomment', 'ID', 'Geopoint', 
           'Scheduledat','Address','ChargeSlots', 'ChargingStatus','Kind', 'Rentalstatus','Slots',],axis=1, inplace=True)

In [0]:
# checking the columns types 
auto.dtypes

#Completeness ~ Null Values Assessment 

In [78]:
#Checking for null values  
auto.isnull().sum().sum()

0

In [0]:
#preview our dataset to ascertain its dropped columns 
auto.head()

#Duplicates 

In [0]:
#Duplicates and Drop them ~ Scheduled at has the most duplicates 14310213  
auto[auto.duplicated(subset = None)]

In [101]:
#Checking if its only city we are dealing with 
auto.City.unique()

array(['Paris'], dtype=object)

#Uniformity ~ Date Type Conversion 

In [0]:
#Uniformity - datetypes as strings 
auto = pd.read_csv('auto_clean.csv', dtype = {'year':str,'month':str,'day':str,'hour':str,'minute':str })

In [0]:
#converting  date_time column 
auto['date_time'] = pd.to_datetime( auto['date_time'])

Displaying Clean Dataset

In [100]:
#DISPLAYING OUR CLEAN DATASET
auto.head()

,Unnamed: 0,Bluecarcounter,Utilibcounter,Utilib1.4counter,City,Postalcode,Publicname,Stationtype,Status,Subscriptionstatus,hour,date_time
2,2,0,0,0,Paris,75020,Paris/Harpignies/3,station,closed,nonexistent,17,2018-04-09 17:46:00
4,4,0,0,0,Paris,75018,Paris/Rachel/1,station,ok,nonexistent,17,2018-04-09 17:46:00
11,11,0,0,1,Paris,75002,Paris/Alexandrie/7,station,ok,nonexistent,17,2018-04-09 17:46:00
15,15,0,0,0,Paris,75019,Paris/Goubet/8,station,ok,nonexistent,17,2018-04-09 17:46:00
16,16,0,0,0,Paris,75013,Paris/Arago/52,station,ok,nonexistent,17,2018-04-09 17:46:00


Saving our clean dataset as csv

In [0]:
#SAVING THE CLEAN DATASET AS CSV
auto.to_csv('auto_clean.csv')

In [0]:
auto.count()

#ANALYSIS ~ Answering Research Questions 

1. What was the most popular hour of the day for picking up a shared electric car (Bluecar) 
in the city of Paris over the month of April 2018?

In [0]:
#ensuring the city is Paris 
auto[auto.City == 'Paris'] 

In [0]:
#this lists the hours for picking the cars 
auto.groupby([auto.date_time.dt.hour])['Bluecarcounter'].sum().sort_values(ascending = False)

2.What is the most popular postal code per hour for returning blue cars? 78005 most popular with 14419 units of cars 

In [0]:
auto.groupby(['Postalcode',auto.date_time.dt.hour,'Bluecarcounter'])['Bluecarcounter'].count().sort_values(ascending = False).head(20)

3. Most Popular Station 

In [0]:
auto.groupby(['Publicname','Bluecarcounter'])['Publicname'].count().sort_values(ascending = False).head(20)

4. Utilib_counter Station at the most popular picking hour for blue cars 

In [0]:
auto.groupby(['Publicname',auto.date_time.dt.hour,'Utilibcounter'])['Utilibcounter'].count().sort_values(ascending = False).head()

5. 